In [2]:
import pandas as pd
import numpy as np

In [3]:
import config
#xdf = pd.read_excel('C:/Users/chvatdi00/Documents/Python Scripts/Spectro/Auswertung_Susanne_Header.xlsx')
#ne = xdf['Unique Code'].nunique()
#print(f"Number of Engines in File: {ne}")

# Das Header File der Auswertung als "Template" einlesen
xdf = pd.read_excel(config.xltemplates, engine='openpyxl')

In [4]:
tmpl = xdf.to_dict(orient='list')
tmpl;

In [5]:
#filename = 'C:/Users/chvatdi00/Documents/Scripts/oildata/problem.csv'
#filename = 'C:/Users/chvatdi00/Documents/Scripts/oildata/spectro_input.csv'
filename = './extract-01_02_2022 12_47_41.csv'
df = pd.read_csv(filename, sep=',', encoding='cp1250')

In [6]:
df;

In [9]:
import os

tempfile = os.getcwd() + '/data/temp.csv'
print(tempfile)
if os.path.exists(tempfile):
    os.remove(tempfile)

in_file = open(filename, "r", encoding='cp1250')
lines = (l for l in in_file)
try:
    out_file = open(tempfile, 'w', encoding='utf-8')
    for line in lines:
        oline = line.replace('\"','')
        out_file.write(oline)
except Exception as err:
    print(str(err))
finally:
    in_file.close()
    out_file.close()


/Users/dieterchvatal/Documents/Anaconda/oildata/data/temp.csv


In [10]:
sdf = pd.read_csv(tempfile, sep=',', index_col=False)

In [11]:
sdf.head(2)

,LaboratoryReference,UniqueCode,Registration,SerialNumber,DateTaken,ReceiptDate,AnalysisDate,TestSuite,SamplingPoint,Status,...,Viscosity at 80°C NA,Viscosity Index,Viscosity Index D2270,Viscosity at -40°C (Brookfield),Water,Water Separation,Water Type,Zinc,Zinc D4951,Zinc D5185
0,GEAG14532,GE0330.0002,-,1075603,10/10/2013,22/10/2013 00:00,25/10/2013 11:31,GEJ-01,-,Normal,...,-,-,-,-,<0.05,-,-,0.0359,-,-
1,GEAG15015,GE0330.0002,-,1075603,21/11/2013,25/11/2013 00:00,29/11/2013 12:38,GEJ-01,-,Normal,...,-,-,-,-,<0.05,-,-,0.0357,-,-


In [ ]:
def lookup(key, tmpl, df):
    if key in tmpl:
        erg = [v for v in tmpl[key] if v in df.columns]
        #print(erg)
        return erg[0] if erg else np.nan
    else:
        return np.nan

In [12]:
tmpl

{'SampleID': ['SampleID', nan, nan, nan],
 'Customer Name': ['Customer Name', nan, nan, nan],
 'Attention': ['Attention', nan, nan, nan],
 'Address 1': ['Address 1', nan, nan, nan],
 'Address 2': ['Address 2', nan, nan, nan],
 'Address 3': ['Address 3', nan, nan, nan],
 'Post Code': ['Post Code', nan, nan, nan],
 'Telephone': ['Telephone', nan, nan, nan],
 'Fax Number': ['Fax Number', nan, nan, nan],
 'Unique Code': ['Unique Code', 'UniqueCode', 'uc', 'uuc'],
 'Customer Reference': ['Customer Reference', nan, nan, nan],
 'Second Customer Reference': ['Second Customer Reference', nan, nan, nan],
 'Serial Number': ['Serial Number', 'SerialNumber', nan, nan],
 'General Description': ['General Description',
  'GeneralDescription',
  nan,
  nan],
 'System Description': ['System Description', 'SystemDescription', nan, nan],
 'Manufacturer': ['Manufacturer', 'Manufacturer', nan, nan],
 'Model': ['Model', 'Model', nan, nan],
 'Oil/Fluid Long Name': ['Oil/Fluid Long Name', 'OilGrade', nan, nan]

In [ ]:
tmpl['SampleID']

In [ ]:
lookup('Unique Code',tmpl,sdf)
lookup('SampleId',tmpl,sdf)

In [ ]:
subkeys = ['SampleID','Unique Code','Oil/Fluid Long Name','Date Taken']
transkeys = [lookup(k, tmpl, sdf) for k in subkeys]
transkeys

In [ ]:
lookup_table = {k:lookup(k, tmpl, sdf) for k in xdf}
lookup_table

In [ ]:
xdf.head(2)

In [ ]:
sdf.head(2)

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

_df = pd.DataFrame([])
for col in xdf.columns:
    if lookup_table[col] in sdf.columns:
        _df[col] = sdf[lookup_table[col]]
    else:
        _df[col] = ""

In [ ]:
_df.head(2)

In [ ]:
import re
date_pattern = "^((\d{2}[-,/,\.]){2}\d{2,4}\s(\d{2}:){1,2}\d{2})(.*)" #matches all date constructs in spectro csv
date_pat = re.compile(date_pattern)
num_pattern = "^[<*]{0,2}([+-]?(\d+(\.\d*)?|\.\d+)([eE][+-]?\d+)?)\??$" #matches all number like constructs in spectro csv
num_pat = re.compile(num_pattern)

def corr2(s):
    # s nur dann behandeln, wenn es ein String ist.  
    if type(s) == str:
        # Leere Felder , Feldee mit '-' und schon vorher leere Felder als None zurücgeben.
        # das bedeutet, dass Excel ein leeres Feld an der Stelle anzeigt.
        if s in ['', '-', None]:
            return None
        if num_pat.match(s):
            try:
                s = float(num_pat.match(s).group(1))
                return s
            except TypeError as err:
                print(f"{s} raised {str(err)}")
                raise
        if date_pat.match(s):
            try:
                s = pd.to_datetime(date_pat.match(s).group(1))
                return s
            except ValueError as err:
                print(f"{s} raised {str(err)}")
                raise
    return s # s has already a different type than str.     

# Umwandels der Datenwerte in Zahlen, z.B. Entfernen von '<' bei '<0.1'
def corr(s):
    # s nur dann behandeln, wenn es ein String ist.  
    if type(s) == str:
        # Leere Felder , Feldee mit '-' und schon vorher leere Felder als None zurücgeben.
        # das bedeutet, dass Excel ein leeres Feld an der Stelle anzeigt.
        if s in ['', '-', None]:
            return None
        if s[0] == '<':
            s = s[1:]
        if date_pat.match(s):
            try:
                s = pd.to_datetime(date_pat.match(s).group(1))
                return s
            except ValueError as err:
                print(f"{s} raised {str(err)}")
                raise
        try:
            s = float(s)
        except ValueError as err:
            pass
            #print(f"float({s}) created Error: {str(err)}")
    return s 

In [ ]:
import re
pattern = "^((\d{2}[-,/,\.]){2}\d{2,4}\s(\d{2}:){1,2}\d{2})(.*)" #matches all date constructs in spectro csv
#pattern_wrong ="^(\d{2}[-,/,\.]){2}\d{2,4}\s(\d{2}:){1,2}\d{2}\s\d{2}[-,/,\.]\d{2}$"
p = re.compile(pattern)
test = ['21/01/2022 07:22 21/01','18/01/2022 11:05:42']
for t in test:
    print(t, p.match(t), p.match(t).group(1))


In [17]:
import re
pattern = "^[<*]{0,2}([+-]?(\d+(\.\d*)?|\.\d+)([eE][+-]?\d+)?)[\?\*]*$" #matches all number like constructs in spectro csv
p = re.compile(pattern)
test = ['<<0.05','<0.05','<0.0001','<1','0.0009','0,0299','1584?','*0.05','**0.05','2228/','?', '4.03*']
for t in test:
    print(f"{t}\t{p.match(t)}\t***{p.match(t).group(1) if p.match(t) else '-'}***")


<<0.05	<re.Match object; span=(0, 6), match='<<0.05'>	***0.05***
<0.05	<re.Match object; span=(0, 5), match='<0.05'>	***0.05***
<0.0001	<re.Match object; span=(0, 7), match='<0.0001'>	***0.0001***
<1	<re.Match object; span=(0, 2), match='<1'>	***1***
0.0009	<re.Match object; span=(0, 6), match='0.0009'>	***0.0009***
0,0299	None	***-***
1584?	<re.Match object; span=(0, 5), match='1584?'>	***1584***
*0.05	<re.Match object; span=(0, 5), match='*0.05'>	***0.05***
**0.05	<re.Match object; span=(0, 6), match='**0.05'>	***0.05***
2228/	None	***-***
?	None	***-***
4.03*	<re.Match object; span=(0, 5), match='4.03*'>	***4.03***


In [ ]:

# die Zellen in Zahlen umwandeln, wo möglich
new_df = _df.applymap(corr2)
resdf = new_df[['SampleID','Unique Code','Oil/Fluid Long Name','Completed Date','Date Taken','Receipt Date','Booked In date','Billing Date','Due Date','Soot by FTIR (% wt)','Oil/Fluid Hours']]
resdf.head(2)